### ✅ Validace a export modelu
#### 🧪 1. Vyhodnocení modelu na testovacích datech

Po dokončení tréninku je důležité model otestovat na nezávislém testovacím datasetu - taková data, která model během tréninku ještě neviděl.

In [ ]:
!python val.py \
  --weights runs/train/yolov12_nano_custom/weights/best.pt \
  --data data/custom_data.yaml \
  --img 640

Tento příkaz vypíše metriky jako:

- *Precision, Recall, mAP@0.5, mAP@0.5:0.95*
- Počet detekovaných objektů
- Chyby podle tříd

### 📦 2. Export modelu

Model lze exportovat do různých formátů pro nasazení:

- PyTorch (**.pt**) – výchozí formát

- **ONNX** (Open Neural Network Exchange)

In [ ]:
!python export.py \
  --weights runs/train/yolov12_nano_custom/weights/best.pt \
  --include onnx \
  --img 640 \
  --dynamic

# Výstup: best.onnx – vhodné pro nasazení v prostředích jako TensorRT, OpenVINO, ONNX Runtime.

- CoreML, TensorRT, OpenVINO, TF SavedModel, TFLite, TF.js

Tyto formáty lze exportovat pomocí --include s odpovídající volbou.

### 🚀 3. Příprava na nasazení

In [ ]:
!python detect.py \
  --weights best.pt \
  --img 640 \
  --source path/to/test/images

- *Optimalizace výkonu:*
Použij **ONNX** + TensorRT pro rychlé inference na GPU.
Použij TFLite pro mobilní zařízení.
Použij TorchScript pro embedded zařízení.

- *Integrace do aplikace:*
**REST API** (např. FastAPI, Flask)
Desktop (např. PyQt, Electron)
Mobilní aplikace (TensorFlow Lite, CoreML)
Web (TensorFlow.js)

### Ukázka jednoduché inference:

#### ✅ Co potřebuješ:
- onnxruntime
- opencv-python
- ONNX model (best.onnx)
- Testovací obrázek (test.jpg)

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
import time

# Cesta k ONNX modelu a vstupnímu obrázku
model_path = "best.onnx"
image_path = "test.jpg"

# Načtení obrázku a předzpracování
img = cv2.imread(image_path)
img_resized = cv2.resize(img, (640, 640))
img_input = img_resized[:, :, ::-1].transpose(2, 0, 1)  # BGR → RGB → CHW
img_input = np.expand_dims(img_input, axis=0).astype(np.float32) / 255.0

# Inicializace ONNX runtime
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name

# Inference
start = time.time()
outputs = session.run(None, {input_name: img_input})
end = time.time()

print(f"Inference time: {end - start:.3f} s")

# Zpracování výstupu
predictions = outputs[0][0]  # [num_detections, 6] → [x1, y1, x2, y2, conf, class]

# Filtrace podle confidence threshold
conf_threshold = 0.25
for det in predictions:
    x1, y1, x2, y2, conf, cls = det
    if conf > conf_threshold:
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, f"{int(cls)} {conf:.2f}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Zobrazení výsledku
cv2.imshow("Detekce", img)
cv2.waitKey(0)
cv2.destroyAllWindows()